In [128]:
import pandas as pd
import nltk as n
import itertools as i
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
"""
Plan
Generate a cleaned, munged csv from sentiment.csv using homework 4
Split munged data into train/test 90/10
Change labels from 0 and 1 to "pos" and "neg"
"""

In [130]:
data = pd.read_csv('clean_munge.csv', delimiter=',', encoding='ISO-8859-1')
smoother = 0.00001

In [131]:
# change 0 and 1 to pos and neg
data["polarity_text"] = data.polarity.apply(lambda p: "neg" if p == 0 else "pos")
data["split"] = data.tweet.apply(lambda t: t.split(" "))
data["no_hashtags"] = data.lowercase.apply(lambda t: t.replace("#", ""))

import nltk
from nltk.stem import PorterStemmer
ps=PorterStemmer()
ps.stem('writing')

def stem(tweet):
    stems = []
    
    tweet_split = tweet.split(" ")
    
    for word in tweet_split:
        stems.append(ps.stem(word))
        
    return stems

data["stems"] = data["lowercase"].apply(stem)

def no_ht_or_mentions(row):
    words = []
    
    for word in row.tweet.split(" "):
        if "#" in word or "@" in word:
            pass
        else:
            words.append(word)
    
    return " ".join(words)

data["no_ht_no_mentions"] = data.apply(no_ht_or_mentions, axis=1)

In [132]:
train, test = train_test_split(data, test_size=0.1)
len(train.index), len(test.index)

(14400, 1600)

In [133]:
data

,Unnamed: 0,polarity,tweet,all_caps,hashtags,mentions,urls,numbers,lowercase,stems,polarity_text,split,no_hashtags,no_ht_no_mentions
0,780018,0,@pingiwingi Poor thing Hope you feel better ...,NaN,NaN,['@pingiwingi'],NaN,NaN,@pingiwingi poor thing hope you feel better ...,"[@pingiwingi, poor, thing, , , hope, you, feel...",neg,"[@pingiwingi, Poor, thing, , , Hope, you, feel...",@pingiwingi poor thing hope you feel better ...,Poor thing Hope you feel better soon.
1,1051249,1,#FF one of my new faves cos y'all know how I a...,['FF'],['#FF'],['@NuttyNewswire'],NaN,NaN,#ff one of my new faves cos y'all know how i a...,"[#ff, one, of, my, new, fave, co, y'all, know,...",pos,"[#FF, one, of, my, new, faves, cos, y'all, kno...",ff one of my new faves cos y'all know how i am...,one of my new faves cos y'all know how I am! -
2,54916,0,last day of #tourdeblono before i leave on the...,NaN,['#tourdeblono'],NaN,NaN,NaN,last day of #tourdeblono before i leave on the...,"[last, day, of, #tourdeblono, befor, i, leav, ...",neg,"[last, day, of, #tourdeblono, before, i, leave...",last day of tourdeblono before i leave on the ...,last day of before i leave on the greyhound at...
3,845262,1,hands up if you love the sum on a Monday morni...,NaN,NaN,NaN,NaN,NaN,hands up if you love the sum on a monday morni...,"[hand, up, if, you, love, the, sum, on, a, mon...",pos,"[hands, up, if, you, love, the, sum, on, a, Mo...",hands up if you love the sum on a monday morni...,hands up if you love the sum on a Monday morni...
4,1066937,1,"@nuthatchgirl Hi Stacie, I just sent you a mes...",NaN,NaN,['@nuthatchgirl'],NaN,NaN,"@nuthatchgirl hi stacie, i just sent you a mes...","[@nuthatchgirl, hi, stacie,, i, just, sent, yo...",pos,"[@nuthatchgirl, Hi, Stacie,, I, just, sent, yo...","@nuthatchgirl hi stacie, i just sent you a mes...","Hi Stacie, I just sent you a message on Etsy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,763700,0,@liamlager i did have an interest in it before...,NaN,NaN,['@liamlager'],NaN,NaN,@liamlager i did have an interest in it before...,"[@liamlag, i, did, have, an, interest, in, it,...",neg,"[@liamlager, i, did, have, an, interest, in, i...",@liamlager i did have an interest in it before...,"i did have an interest in it before i saw it, ..."
15996,1137306,1,There's nothing in the world that mommy's kiss...,NaN,NaN,NaN,NaN,NaN,there's nothing in the world that mommy's kiss...,"[there', noth, in, the, world, that, mommy', k...",pos,"[There's, nothing, in, the, world, that, mommy...",there's nothing in the world that mommy's kiss...,There's nothing in the world that mommy's kiss...
15997,856219,1,@Acousticore though we're gonna see it tomorro...,NaN,NaN,['@Acousticore'],NaN,['3'],@acousticore though we're gonna see it tomorro...,"[@acousticor, though, we'r, gonna, see, it, to...",pos,"[@Acousticore, though, we're, gonna, see, it, ...",@acousticore though we're gonna see it tomorro...,though we're gonna see it tomorrow i guess - o...
15998,1172092,1,Professor said &quot;you could have knocked me...,NaN,NaN,NaN,NaN,['2009'],professor said &quot;you could have knocked me...,"[professor, said, &quot;you, could, have, knoc...",pos,"[Professor, said, &quot;you, could, have, knoc...",professor said &quot;you could have knocked me...,Professor said &quot;you could have knocked me...


### Estimate P(T)

In [134]:
from collections import Counter as ctr

p_t_estimate = ctr(train.polarity_text)
p_t_total = len(train)

In [135]:
p_t_estimate['neg'], p_t_estimate['pos']

(7196, 7204)

In [136]:
def Pt(T):
    return p_t_estimate[T] / p_t_total

In [137]:
Pt('neg'), Pt('pos')

(0.49972222222222223, 0.5002777777777778)

### Estimate P(W)

In [138]:
tweet_words = [w for s in train.split for w in s]
p_w_estimate = ctr(tweet_words)
p_w_total = len(tweet_words)

In [139]:
def Pw(W):
    if W not in p_w_estimate: return smoother
    return p_w_estimate[W] / p_w_total

In [140]:
Pw('the')

0.021387202399610353

In [141]:
np.sum([Pw(w) for w in set(tweet_words)])

1.0

### Estimate P(W|T)

In [142]:
p_w_t_estimate = {}
p_w_t_totals = {}

for t in set(train.polarity_text):
    sub_frame = train[train.polarity_text == t]
    sub_words = [w for s in sub_frame.split for w in s]
    p_w_t_estimate[t] = ctr(sub_words)
    p_w_t_totals[t] = len(sub_words)

In [143]:
def Pwt(W, T):
    if W not in p_w_t_estimate[T]: return smoother
    return p_w_t_estimate[T][W] / p_w_t_totals[T]

### Bayes' Rule

In [144]:
def Ptw(T, W):
    return Pwt(W, T) * Pt(T) / Pw(W)

In [145]:
def Pe(E):
    result = {}
    for t in set(train.polarity_text):
        result[t] = np.prod([Ptw(t, word) for word in E])
    return result

In [146]:
Pe("today has been horrible".split(" ")), Pe("Today has been horrible".split(" "))

({'pos': 0.00342866987624089, 'neg': 0.20868887176710693},
 {'pos': 0.00258572388856779, 'neg': 0.24149146202558178})

### Baseline

In [147]:
def score(df, col):
    results_col = col + "_result"
    top_col = col + "_top"
    df[results_col] = df[col].apply(Pe)
    df[top_col] = df[results_col].apply(lambda x: max(x, key=x.get))
    return sum(df.polarity_text == df[top_col]) / len(df)

score(df=test, col="split")

c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.76125

### Scores

In [148]:
score(df=test, col="lowercase")

c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.469375

In [149]:
score(df=test, col="no_hashtags")

c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.469375

In [150]:
score(df=test, col="stems")

c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.734375

### Open ended approach

In [153]:
"""
I am going to remove all hashtag words and mentions from each tweet.
Then, run the classifier on the new tweets and get a score.
"""

score(df=test, col="no_ht_no_mentions")

c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\steven\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


0.478125

### Questions

The baseline:
    
    The types of tweets, negative or positive, were evenly probable at about 50% each.

Classifier accuracy of split text:
    
    .76
    
Classifier accuracy of lower-cased, split text
    
    .47
    
Classifier accuracy of the lower-cased, split text with hashtag symbols removed

    .47
    
Classifier accuracy of lower-cased, split text with stemmed words

    .73
    
Open ended approach classifier accuracy
    
    .48
    
    My hypothesis was that hashtags and mentions are mostly "white noise" in the tweets.
    Hashtags and mentions can be evenly used for both negative and positive tweets which is why I opted to remove them.
    But my classifier accuracy on those tweets in the lowest of all classifier accuracies. 
    This leads me to believe that hashtags and/or mentions carry a significant sentiment value and should not be
    disregarded.